<a href="https://colab.research.google.com/github/Ruby-L-i-n/CSDS234_FinalProject/blob/main/Files%20for%20Content-Based%20Recommendation%20Model/ContentBasedSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import heapq
import random
import json
import matplotlib.pyplot as pyplot

import seaborn as sns
sns.set(style='whitegrid')
pd.set_option('display.width', 1500)
pd.set_option('display.max_columns', 100)

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


###Reading Json file and data cleaning

In [ ]:
#select 100 random playlist since the data is too large.
np.random.seed(seed = 19) #a random seed to ensure calculation is the same across methods. change to generate a different set of random numbers
numbers = np.arange(0,1000,1) #a range of numbers from 1 to 1000
sample = np.random.choice(numbers, size=100, replace=True) #randomly select 100 playlists
print(sample)

[605 622 757 354 936  19 308 330 671 834 440 534 162 901 609 266 342 306
 364 167 783 231 976 807 327 777 203 201 772 853 869 999 988 849 369 194
  53 291 313  55 872 188 869 325 416 817 708 438 220 510 465 780 467 433
 941 212 273 467 837 409 435 475 183 905 779 733 612 379 446 956 246 896
 233 803 114 191 636 671  32 575 778 727 618 556 474 789 707 597 954 907
 169  88 816 634 839 449 872 126 393 372]


In [ ]:
with open("/content/drive/Shareddrives/CSDS 234 Final Project/spotify_million_playlist_dataset/data/mpd.slice.0-999.json") as f:
    data = json.load(f)

flattened_playlists = []
for playlist in data['playlists']:
    playlist_info = {
        'pid': playlist['pid'],
        'name': playlist.get('name', None),
        'collaborative': playlist['collaborative'],
        'modified_at': playlist['modified_at'],
        'num_tracks': playlist['num_tracks'],
        'num_albums': playlist['num_albums'],
        'num_followers': playlist['num_followers']
    }

    for track in playlist['tracks']:
        track_info = {
            'playlist_pid': playlist['pid'],
            'track_pos': track['pos'],
            'track_name': track['track_name'],
            'track_uri': track['track_uri'],
            'artist_name': track['artist_name'],
            'artist_uri': track['artist_uri'],
            'album_name': track['album_name'],
            'album_uri': track['album_uri'],
            'duration_ms': track['duration_ms']
        }
        flattened_playlists.append({**playlist_info, **track_info})

# Create a Pandas DataFrame
df = pd.DataFrame(flattened_playlists)
df.head(10)

,pid,name,collaborative,modified_at,num_tracks,num_albums,num_followers,playlist_pid,track_pos,track_name,track_uri,artist_name,artist_uri,album_name,album_uri,duration_ms
0,0,Throwbacks,false,1493424000,52,47,1,0,0,Lose Control (feat. Ciara & Fat Man Scoop),spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Missy Elliott,spotify:artist:2wIVse2owClT7go1WT98tk,The Cookbook,spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863
1,0,Throwbacks,false,1493424000,52,47,1,0,1,Toxic,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,Britney Spears,spotify:artist:26dSoYclwsYLMAKD3tpOr4,In The Zone,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800
2,0,Throwbacks,false,1493424000,52,47,1,0,2,Crazy In Love,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Beyoncé,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Dangerously In Love (Alben für die Ewigkeit),spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933
3,0,Throwbacks,false,1493424000,52,47,1,0,3,Rock Your Body,spotify:track:1AWQoqb9bSvzTjaLralEkT,Justin Timberlake,spotify:artist:31TPClRtHm23RisEBtV3X7,Justified,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266
4,0,Throwbacks,false,1493424000,52,47,1,0,4,It Wasn't Me,spotify:track:1lzr43nnXAijIGYnCT8M8H,Shaggy,spotify:artist:5EvFsr3kj42KNv97ZEnqij,Hot Shot,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600
5,0,Throwbacks,false,1493424000,52,47,1,0,5,Yeah!,spotify:track:0XUfyU2QviPAs6bxSpXYG4,Usher,spotify:artist:23zg3TcAtWQy7J6upgbUnj,Confessions,spotify:album:0vO0b1AvY49CPQyVisJLj0,250373
6,0,Throwbacks,false,1493424000,52,47,1,0,6,My Boo,spotify:track:68vgtRHr7iZHpzGpon6Jlo,Usher,spotify:artist:23zg3TcAtWQy7J6upgbUnj,Confessions,spotify:album:1RM6MGv6bcl6NrAG8PGoZk,223440
7,0,Throwbacks,false,1493424000,52,47,1,0,7,Buttons,spotify:track:3BxWKCI06eQ5Od8TY2JBeA,The Pussycat Dolls,spotify:artist:6wPhSqRtPu1UhRCDX5yaDJ,PCD,spotify:album:5x8e8UcCeOgrOzSnDGuPye,225560
8,0,Throwbacks,false,1493424000,52,47,1,0,8,Say My Name,spotify:track:7H6ev70Weq6DdpZyyTmUXk,Destiny's Child,spotify:artist:1Y8cdNmUJH7yBTd9yOvr5i,The Writing's On The Wall,spotify:album:283NWqNsCA9GwVHrJk59CG,271333
9,0,Throwbacks,false,1493424000,52,47,1,0,9,Hey Ya! - Radio Mix / Club Mix,spotify:track:2PpruBYCo4H7WOBJ7Q2EwM,OutKast,spotify:artist:1G9G7WwrXka3Z1r7aIDjI7,Speakerboxxx/The Love Below,spotify:album:1UsmQ3bpJTyK6ygoOOjG1r,235213


In [ ]:
# #clean the data by only using the random 100 playlists
# clean_df = df.loc[df["pid"].isin(sample)]
# clean_df.head()

Before we start it is usful to gain some insights on the data we are working with.

In [ ]:
# print("total number of songs: ", len(set(clean_df["track_name"])))

In [ ]:
# pyplot.hist(clean_df['pid'].value_counts())
# pyplot.title("Distribution of number of songs in our 100 Playlists")
# pyplot.ylabel("Count")
# pyplot.xlabel("Number of songs in a playlist")
# print(clean_df['pid'].value_counts()[0:10])
# pyplot.show()

In [ ]:
# # Checking which artists are featured most across all playlists
# values = list(clean_df['artist_name'].value_counts())
# names = clean_df['artist_name'].value_counts().index.tolist()

# ax = sns.barplot(y=names[0:10], x=values[0:10])
# ax.set_title('Artists with most featured songs across all playlists')
# ax.set_xlabel('Total songs across playlists')

# CONTENT BASED RECOMMENDATION SYSTEM

In [ ]:
# Duplicates
duplicates = df[df.duplicated(['artist_name', 'track_name'])]
duplicates_Drake = duplicates[duplicates['artist_name'] == 'Drake']
sorted = duplicates_Drake.sort_values(by=['track_name'], ascending=True)
sorted

,pid,name,collaborative,modified_at,num_tracks,num_albums,num_followers,playlist_pid,track_pos,track_name,track_uri,artist_name,artist_uri,album_name,album_uri,duration_ms
47631,704,Drake,false,1498348800,220,96,1,704,102,0 To 100 / The Catch Up,spotify:track:4jtyUzZm9WLc2AdaJ1dso7,Drake,spotify:artist:3TVXtAsR1Inumwj472S9r4,0 To 100 / The Catch Up,spotify:album:5OkxLN5XaE9rLbgK2FuKBE,275226
50477,751,xmas party,false,1456185600,150,136,2,751,16,0 To 100 / The Catch Up,spotify:track:4jtyUzZm9WLc2AdaJ1dso7,Drake,spotify:artist:3TVXtAsR1Inumwj472S9r4,0 To 100 / The Catch Up,spotify:album:5OkxLN5XaE9rLbgK2FuKBE,275226
12463,193,🔥🔥🔥🔥,false,1442102400,14,13,1,193,3,0 To 100 / The Catch Up,spotify:track:4jtyUzZm9WLc2AdaJ1dso7,Drake,spotify:artist:3TVXtAsR1Inumwj472S9r4,0 To 100 / The Catch Up,spotify:album:5OkxLN5XaE9rLbgK2FuKBE,275226
30049,454,issa playlist,false,1509321600,159,82,1,454,116,0 To 100 / The Catch Up,spotify:track:4jtyUzZm9WLc2AdaJ1dso7,Drake,spotify:artist:3TVXtAsR1Inumwj472S9r4,0 To 100 / The Catch Up,spotify:album:5OkxLN5XaE9rLbgK2FuKBE,275226
13479,211,My music,false,1492646400,79,67,2,211,73,0 To 100 / The Catch Up,spotify:track:4jtyUzZm9WLc2AdaJ1dso7,Drake,spotify:artist:3TVXtAsR1Inumwj472S9r4,0 To 100 / The Catch Up,spotify:album:5OkxLN5XaE9rLbgK2FuKBE,275226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47532,704,Drake,false,1498348800,220,96,1,704,3,Wu-Tang Forever,spotify:track:6KziiQUOoCmC7Kc7Rv4jar,Drake,spotify:artist:3TVXtAsR1Inumwj472S9r4,Nothing Was The Same,spotify:album:2gXTTQ713nCELgPOS0qWyt,217680
53655,806,2K14,false,1414800000,143,121,3,806,18,Wu-Tang Forever,spotify:track:6KziiQUOoCmC7Kc7Rv4jar,Drake,spotify:artist:3TVXtAsR1Inumwj472S9r4,Nothing Was The Same,spotify:album:2gXTTQ713nCELgPOS0qWyt,217680
52079,776,ha,false,1451952000,35,15,1,776,17,Wu-Tang Forever,spotify:track:6KziiQUOoCmC7Kc7Rv4jar,Drake,spotify:artist:3TVXtAsR1Inumwj472S9r4,Nothing Was The Same,spotify:album:2gXTTQ713nCELgPOS0qWyt,217680
49906,743,New ish,false,1509148800,245,168,1,743,89,You & The 6,spotify:track:2YaDRtIlQiZ5WDDB2YuEOC,Drake,spotify:artist:3TVXtAsR1Inumwj472S9r4,If You're Reading This It's Too Late,spotify:album:0ptlfJfwGTy0Yvrk14JK1I,264906


In [ ]:
#Removing duplicates
clean_df = df.drop_duplicates(['track_uri'])

Drake_tracks = cleaned_df[cleaned_df['artist_name']=='Drake']
Drake_tracks.sort_values(by='track_name', ascending=True)

,pid,name,collaborative,modified_at,num_tracks,num_albums,num_followers,playlist_pid,track_pos,track_name,track_uri,artist_name,artist_uri,album_name,album_uri,duration_ms
3595,62,hype,false,1505779200,33,32,2,62,28,0 To 100 / The Catch Up,spotify:track:4jtyUzZm9WLc2AdaJ1dso7,Drake,spotify:artist:3TVXtAsR1Inumwj472S9r4,0 To 100 / The Catch Up,spotify:album:5OkxLN5XaE9rLbgK2FuKBE,275226
20977,335,Litty,false,1507939200,143,94,2,335,115,10 Bands,spotify:track:12d5QFwzh60IIHlsSnAvps,Drake,spotify:artist:3TVXtAsR1Inumwj472S9r4,If You're Reading This It's Too Late,spotify:album:0ptlfJfwGTy0Yvrk14JK1I,177733
25258,397,Random,false,1468281600,22,17,1,397,12,30 for 30 Freestyle,spotify:track:1DmnEYXa4WfbdhAPwNzgD8,Drake,spotify:artist:3TVXtAsR1Inumwj472S9r4,What A Time To Be Alive,spotify:album:1ozpmkWcCHwsQ4QTnxOOdT,253934
47539,704,Drake,false,1498348800,220,96,1,704,10,305 To My City,spotify:track:1y9g5UW31xBqvOi10tcXGX,Drake,spotify:artist:3TVXtAsR1Inumwj472S9r4,Nothing Was The Same,spotify:album:2gXTTQ713nCELgPOS0qWyt,255853
18192,284,lit,false,1502150400,73,51,1,284,9,6 God,spotify:track:3a8tAZFJxlmBwOtrf5L1oC,Drake,spotify:artist:3TVXtAsR1Inumwj472S9r4,If You're Reading This It's Too Late,spotify:album:0ptlfJfwGTy0Yvrk14JK1I,180666
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47698,704,Drake,false,1498348800,220,96,1,704,169,Weston Road Flows,spotify:track:4PA16FAl8LPmFmOhARawdV,Drake,spotify:artist:3TVXtAsR1Inumwj472S9r4,Views,spotify:album:40GMAhriYJRO1rsY4YdrZb,253533
10775,161,English,false,1476403200,161,118,2,161,158,With You,spotify:track:3aJkV6DUTSCqOwVwaBDG9B,Drake,spotify:artist:3TVXtAsR1Inumwj472S9r4,Views,spotify:album:40GMAhriYJRO1rsY4YdrZb,195053
28764,439,housewarming,false,1438128000,174,116,2,439,86,Worst Behavior,spotify:track:6oF3Es1YzzmLKjGBfThUvD,Drake,spotify:artist:3TVXtAsR1Inumwj472S9r4,Nothing Was The Same,spotify:album:2gXTTQ713nCELgPOS0qWyt,270186
28763,439,housewarming,false,1438128000,174,116,2,439,85,Wu-Tang Forever,spotify:track:6KziiQUOoCmC7Kc7Rv4jar,Drake,spotify:artist:3TVXtAsR1Inumwj472S9r4,Nothing Was The Same,spotify:album:2gXTTQ713nCELgPOS0qWyt,217680


In [ ]:
!pip install spotipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 4.2 MB/s eta 0:00:00


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [ ]:
from google.colab import userdata
client_id = userdata.get('client_id')
client_secret = userdata.get('client_secret')

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [ ]:
playlist_link = "https://open.spotify.com/playlist/37i9dQZF1DX1dvMSwf27JO?si=a3f631ae09d74ee9"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]

for track in sp.playlist_tracks(playlist_URI)["items"]:
    #URI
    track_uri = track["track"]["uri"]
    print(track_uri)

    #Track name
    track_name = track["track"]["name"]
    print(track_name)

    #Main Artist
    artist_uri = track["track"]["artists"][0]["uri"]
    artist_info = sp.artist(artist_uri)

    #Name, popularity, genre
    artist_name = track["track"]["artists"][0]["name"]
    artist_pop = artist_info["popularity"]
    artist_genres = artist_info["genres"]

    #Album
    album = track["track"]["album"]["name"]

    #Popularity of the track
    track_pop = track["track"]["popularity"]

In [ ]:
sp.audio_features("2GAhgAjOhEmItWLfgisyOn")[0]

[{'danceability': 0.698,
  'energy': 0.546,
  'key': 0,
  'loudness': -7.234,
  'mode': 0,
  'speechiness': 0.0559,
  'acousticness': 0.425,
  'instrumentalness': 0.00314,
  'liveness': 0.16,
  'valence': 0.335,
  'tempo': 145.044,
  'type': 'audio_features',
  'id': '2GAhgAjOhEmItWLfgisyOn',
  'uri': 'spotify:track:2GAhgAjOhEmItWLfgisyOn',
  'track_href': 'https://api.spotify.com/v1/tracks/2GAhgAjOhEmItWLfgisyOn',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2GAhgAjOhEmItWLfgisyOn',
  'duration_ms': 181080,
  'time_signature': 4}]